# ** Cantilever Beam:**

**Boundary Conditions:**   ```v[x = 0] = 0; dv/dx[x = 0] = 0```

---
Point Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, EI, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    # EI =  2000     Flexural rigidity (N·m^2)
    K = 1/EI         # Assigning value of 1/EI to K : It reduces the time for run of the code
    P  = -100.0      # Point load at the free end (N)
    w  =  0.5        # Relaxation factor
    inv_dx = 1/dx
    inv_dx2 = inv_dx**2

    # init_guess == 'uniform': For Point Loading
    v = 0.0001 * np.sign(P) * x[:]                         # Better than both. Cons: Jacobian becomes singular for N > 372.
    v[0] = 0
    print()
    print(v)

    # Moment Equation
    M = np.zeros(N+1)
    for i in range(N+1):
        M[i] = -P*(L - x[i])

    # Residual Function | It is same for both Point and Continuous Load
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                R[i] = ((-3*v_full[i]+4*v_full[i+1]-v_full[i+2])*inv_dx)*(1/2)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                    J[i, i], J[i, i+1], J[i, i+2] =  0, (2)*(inv_dx), (-0.5)*(inv_dx)
            elif i == N:
                J[i, i-3] = -inv_dx2
                J[i, i-2] =  4*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2*inv_dx2
                J[i, i+1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 +  (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, M)
        print(np.linalg.norm(R))
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, M)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0

    return x, v_full, L, P, K, inv_dx, inv_dx2


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 150
EI = 2000
x, v, L, P, K, inv_dx, inv_dx2 = newton_raphson_system(N, EI)
print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, P*x**2/(6*EI) * (3*L - x), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()

# Create Pandas DataFrame
df = pd.DataFrame({
    'x': x,
    'Newton-Raphson Solution (point load)': v,
    'Euler-Bernoulli Solution (point load)': ((P*x**2*(3*L - x))*K)*(1/6),
})
# Create interactive plot using Plotly
fig = df.iplot(
    kind='line',
    x='x',
    y=[
        'Newton-Raphson Solution (point load)',
        'Euler-Bernoulli Solution (point load)',
    ],
    asFigure=True
)
fig.show()



# Point load(With Table of Error)

In [ ]:
import numpy as np

def newton_raphson_system(N, EI, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    # EI =  2000     Flexural rigidity (N·m^2)
    K = 1/EI         # Assigning value of 1/EI to K : It reduces the time for run of the code
    P  = -100.0      # Point load at the free end (N)
    w  =  0.5        # Relaxation factor
    inv_dx = 1/dx
    inv_dx2 = inv_dx**2

    # init_guess == 'uniform': For Point Loading
    v = 0.0001 * np.sign(P) * x[:]                         # Better than both. Cons: Jacobian becomes singular for N > 372.
    v[0] = 0
    print()
    print(v)

    # Moment Equation
    M = np.zeros(N+1)
    for i in range(N+1):
        M[i] = -P*(L - x[i])

    # Residual Function | It is same for both Point and Continuous Load
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                R[i] = ((-3*v_full[i]+4*v_full[i+1]-v_full[i+2])*inv_dx)*(1/2)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                    J[i, i], J[i, i+1], J[i, i+2] =  0, (2)*(inv_dx), (-0.5)*(inv_dx)
            elif i == N:
                J[i, i-3] = -inv_dx2
                J[i, i-2] =  4*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2*inv_dx2
                J[i, i+1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 +  (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, M)
        print(np.linalg.norm(R))
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, M)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0

    return x, v_full, L, P, K, inv_dx, inv_dx2


# Solve and visualize
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''

L = 1.0
N = 150
x = np.linspace(0, L, N+1)
P = -100.0
EI_arr = [20, 30, 40, 50, 100, 500, 1000, 2000]  # Array of EI values to iterate over
def euler_bernoulli_deflection(x, L, P, EI):
    return (P*x**2*(3*L - x))/(6*EI)

# Storage for plotting
table = [[], [], [], []]

for EI in EI_arr:
    print(f"Solving for EI = {EI}")
    x, v, L, P, K, inv_dx, inv_dx2 = newton_raphson_system(N, EI)
    v_euler = euler_bernoulli_deflection(x, L, P, EI)

    # Compute deflection values at the tip
    v_euler_at_tip = v_euler[-1]
    v_NR_at_tip = v[-1]

    # Error Calculation
    final_error = (v_NR_at_tip - v_euler_at_tip) * 100 / v_euler_at_tip

    # Store results in table array
    table[0].append(EI)
    table[1].append(v_euler_at_tip)
    table[2].append(v_NR_at_tip)
    table[3].append(final_error)

print(f"Table Array:\n{table}")

# EI values and corresponding results
EI_values = table[0]
v_euler_at_tip_values = table[1]
v_NR_at_tip_values = table[2]
error_values = table[3]

# Plot 1: Error Variation with EI
plt.figure(figsize=(8, 5))
plt.plot(EI_values, error_values, marker='o', linestyle='-', color='r', label="Error (%)")
plt.xlabel("Flexural Rigidity (EI)")
plt.ylabel("Relative Percentage Error (%)")
plt.title("Error Variation with EI (At the Free End)")
plt.xscale("log")     # Log scale for better visualization if EI varies significantly
plt.xlim(left=0)      # Ensures that the x-axis starts from 0
plt.grid(True, which="both", linestyle="--", linewidth=0.5)

# Minor ticks and tick placement inside
plt.minorticks_on()
plt.tick_params(axis="both", which="both", direction="in")

# Ensure exactly 2 minor ticks between major ticks on y-axis
ax = plt.gca()
major_ticks = ax.yaxis.get_majorticklocs()
if len(major_ticks) > 1:
    major_step = np.diff(major_ticks).mean()  # Compute mean spacing of major ticks
    ax.yaxis.set_minor_locator(MultipleLocator(major_step / 3))

plt.legend()
plt.show()


# Plot 2: v_euler_at_tip and v_NR_at_tip vs EI
plt.figure(figsize=(8, 5))
plt.plot(EI_values, v_euler_at_tip_values, marker='s', linestyle='-', label="v_euler_at_tip")
plt.plot(EI_values, v_NR_at_tip_values, marker='^', linestyle='-', label="v_NR_at_tip")
plt.xlabel("Flexural Rigidity (EI)")
plt.ylabel("Deflection at the Free End")
plt.title("Deflection at the Free End vs EI")
plt.xscale("log")     # Log scale if EI has a large range
plt.xlim(left=0)      # Ensures that the x-axis starts from 0
plt.grid(True, which="both", linestyle="--", linewidth=0.5)

# Minor ticks and tick placement inside
plt.minorticks_on()
plt.tick_params(axis="both", which="both", direction="in")

# Ensure exactly 2 minor ticks between major ticks on y-axis
ax = plt.gca()
major_ticks = ax.yaxis.get_majorticklocs()
if len(major_ticks) > 1:
    major_step = np.diff(major_ticks).mean()  # Compute mean spacing of major ticks
    ax.yaxis.set_minor_locator(MultipleLocator(major_step / 3))

plt.legend(loc='lower right')
plt.show()



---
Comparison of Point Load v/s Distributed Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.
    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000     # Flexural rigidity (N·m^2)
    K = 1/EI         # Assigning value of 1/EI to K : It reduces the time for run of the code
    P  = -100.0      # Point load at the free end (N)
    W = -100.0       # Continuous Load (N/m) acting downward
    w  =  0.5        # Relaxation factor
    inv_dx = 1/dx
    inv_dx2 = inv_dx**2

    # Initial guess -- continuos load


    # init_guess == 'uniform': For Continuos Loading
    v_cont = 0.0001 * np.sign(W) * x[:]                         # Better than both. Cons: Jacobian becomes singular for N > 372.
    v_cont[0] = 0

    # init_guess == 'uniform': For Point Loading
    v_point = 0.0001 * np.sign(P) * x[:]                         # Better than both. Cons: Jacobian becomes singular for N > 372.
    v_point[0] = 0

    # Moment Equation
    M_point = np.zeros(N+1)
    M_cont = np.zeros(N+1)
    for i in range(N+1):
        M_cont[i] = W*L*x[i] - (W*(x[i]**2))/2 - (W*(L**2))/2
        M_point[i] = -P*(L-x[i])

    # Residual Function | It is same for both Point and Continuous Load
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                R[i] = ((-3*v_full[i]+4*v_full[i+1]-v_full[i+2])*inv_dx)*(1/2)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                    J[i, i], J[i, i+1], J[i, i+2] =  0, (2)*(inv_dx), (-0.5)*(inv_dx)
            elif i == N:
                J[i, i-3] = -inv_dx2
                J[i, i-2] =  4*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2*inv_dx2
                J[i, i+1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 +  (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration -- continuous load
    for iteration in range(max_iter):
        R_cont = residual(v_cont, M_cont)
        print(np.linalg.norm(R_cont))
        if np.linalg.norm(R_cont) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J_cont = jacobian(v_cont, M_cont)
        try:
            # Attempt to solve the system
            delta_v_cont = np.linalg.solve(J_cont, -R_cont)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v_cont = np.linalg.lstsq(J_cont, -R_cont, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_cont += w*delta_v_cont

        if np.linalg.norm(delta_v_cont) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Newton-Raphson iteration -- point load
    for iteration in range(max_iter):
            R_point = residual(v_point, M_point)
            print(np.linalg.norm(R_point))
            if np.linalg.norm(R_point) < tol:
                print(f"\nConverged in {iteration+1} iterations.\n")
                break

            J_point = jacobian(v_point, M_point)
            try:
                # Attempt to solve the system
                delta_v_point = np.linalg.solve(J_point, -R_point)
            except np.linalg.LinAlgError:
                # If singular, use pseudo-inverse
                print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
                delta_v_point = np.linalg.lstsq(J_point, -R_point, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

            v_point += w*delta_v_point

            if np.linalg.norm(delta_v_point) < tol:
                print(f"\nConverged in {iteration+1} iterations.\n")
                break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full_cont = np.zeros(N+1)
    v_full_cont[:] = v_cont
    v_full_cont[0] = 0

    v_full_point = np.zeros(N+1)
    v_full_point[:] = v_point
    v_full_point[0] = 0

    return x, v_full_point, v_full_cont, L, P, W, EI, K, inv_dx, inv_dx2


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250

x, v_point, v_cont, L, P, W, EI, K, inv_dx, inv_dx2 = newton_raphson_system(N)
# print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, P*x**2/(6*EI) * (3*L - x), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()



# Create Pandas DataFrame
df = pd.DataFrame({
    'x': x,
    'Newton-Raphson Solution (point load)': v_point,
    'Newton-Raphson Solution (continuous load)': v_cont,
    'Euler-Bernoulli Solution (point load)': ((P*x**2*(3*L - x))*K)*(1/6),
    'Euler-Bernoulli Solution (continuous load)': - ((W*L*x**3)*K)*(1/6) + ((W*x**4)*K)*(1/24) + ((W*L**2*x**2)*K)*(1/4)    # I take -ve value for deflection in "Euler-Bernoulli Solution (continuous load)"
})



# Create interactive plot using Plotly
fig = df.iplot(
    kind='line',
    x='x',
    y=[
        'Newton-Raphson Solution (point load)',
        'Newton-Raphson Solution (continuous load)',
        'Euler-Bernoulli Solution (point load)',
        'Euler-Bernoulli Solution (continuous load)'
    ],
    asFigure=True
)
fig.show()


---
## Implementation of Point Load and Uniform Continuous Load
* At any number of points and range along the beam
---

In [ ]:
import numpy as np

def newton_raphson_system(N, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    # valid_init_guesses = ['random', 'uniform', 'zeros']
    # valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    # if init_guess not in valid_init_guesses:
    #     raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    # if dv_dx_at_firstPoint not in valid_dv_dx_methods:
    #     raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    K = 1/EI         # Assigning value of 1/EI to K : It reduces the time for run of the code
    P  = -100.0      # Point load at the free end (N)
    W  = -100.0      # Continuous Load (N/m)
    w  =  0.5        # Relaxation factor
    inv_dx = 1/dx
    inv_dx2 = inv_dx**2
    # Initial guess

    v = np.zeros(N+1)          # Good guess. Achieves Middle Ground.
    v[0] = 0

    # Getting the Point Loads
    def get_point_loads():
        n = int(input("Enter the number of point loads: "))
        point_loads = []
        for i in range(n):
            pos, mag = map(float, input(f"Enter position and magnitude of point load {i+1} (space-separated): ").split())
            point_loads.append((pos, mag))
        return point_loads

    # Getting the Continuous Loads
    def get_continuous_loads():
        m = int(input("Enter the number of continuous loads: "))
        continuous_loads = []
        for i in range(m):
            start, end, intensity = map(float, input(f"Enter start position, end position, and intensity of continuous load {i+1} (space-separated): ").split())
            continuous_loads.append((start, end, intensity))
        return continuous_loads

    # Generating the Moment Equation
    def moment_equation(L, point_loads, continuous_loads, N):
        x_vals = np.linspace(0, L, N+1)
        moment_values = np.zeros(N+1)

        for i, x in enumerate(x_vals):   #
            M = 0
            for pos, mag in point_loads:
                if not (0 <= pos <= L):
                    raise ValueError(f"Point load at {pos} exceeds beam length {L}.")
                if x <= pos:
                    M += mag * (x - pos)

            for start, end, intensity in continuous_loads:
                if not (0 <= start <= end <= L):
                    raise ValueError(f"Continuous load from {start} to {end} exceeds beam length {L}.")

                if x <= start:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x
                elif start < x < end:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x - intensity * (x - start)**2 / 2
                else:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x - intensity * (end - start) * (x - (start + end) / 2)

            moment_values[i] = M

        return moment_values

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                R[i] = ((-3*v_full[i]+4*v_full[i+1]-v_full[i+2])*inv_dx)*(1/2)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                    J[i, i], J[i, i+1], J[i, i+2] =  0, (2)*(inv_dx), (-0.5)*(inv_dx)
            elif i == N:
                J[i, i-3] = -inv_dx2
                J[i, i-2] =  4*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2*inv_dx2
                J[i, i+1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 +  (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i+1]-2*v_full[i-1])
        return J

    point_loads = get_point_loads()
    continuous_loads = get_continuous_loads()
    moment_values = moment_equation(L, point_loads, continuous_loads, N)
    print()

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, moment_values)
        print(f"Residual = {np.linalg.norm(R)}")
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, moment_values)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"Jacobian is singular for iteration {iteration+1}. Using pseudo-inverse.\n")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0

    return x, v_full, L, P, W, EI, K, inv_dx, inv_dx2


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250
x, v, L, P, W, EI, K, inv_dx, inv_dx2 = newton_raphson_system(N)
# print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, P*x**2/(6*EI) * (3*L - x), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()

# Create Pandas DataFrame
df = pd.DataFrame({
    'x': x,
    'Newton-Raphson Solution ': v,
    'Euler-Bernoulli Solution (point load)': ((P*x**2*(3*L - x))*K)*(1/6),
    'Euler-Bernoulli Solution (continuous load)': - ((W*L*x**3)*K)*(1/6) + ((W*x**4)*K)*(1/24) + ((W*L**2*x**2)*K)*(1/4)    # I take -ve value for deflection in "Euler-Bernoulli Solution (continuous load)"
})


# Create interactive plot using Plotly
fig = df.iplot(
    kind='line',
    x='x',
    y=[
        'Newton-Raphson Solution ',
        'Euler-Bernoulli Solution (point load)',
        'Euler-Bernoulli Solution (continuous load)'
    ],
    asFigure=True
)
fig.show()
